# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [15]:
TRIAL_LABELS_DF = pd.read_excel("./proc/rce2_finalized_trial_labels.xlsx")

In [16]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_04_spectral_and_sleap.pkl")

In [17]:
# CHANNEL_MAP_AND_ALL_TRIALS_DF = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")
# MERGED_TRIALS_AND_VIDEO = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")

In [18]:
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}

In [19]:
BIN_TO_VELOCITY = {0: "0 to 2.5cm/s", 1: "2.5 to 5cm/s", 2: "5 to 10 cm/s", 3: "10cm/s+"}

In [20]:
ALL_BANDS = ["theta", "beta", "gamma"]
BAND_TO_FREQ = {"theta": {"low_freq": 6, "high_freq": 11}, "beta": {"low_freq": 20, "high_freq": 31}, "gamma": {"low_freq": 30, "high_freq": 51}}
BAND_TO_FREQ = {"theta": (6,11), "beta": (20,31), "gamma": (30,51)}

In [21]:
GROUPINGS = "trial_outcome"
# GROUPINGS = "competition_closeness"

In [22]:
FONTSIZE=20

In [23]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

In [24]:
def generate_pairs(lst):
    pairs = []
    n = len(lst)
    for i in range(n):
        for j in range(i+1, n):
            pairs.append((lst[i], lst[j]))
    return pairs

In [25]:
def nested_dict():
    return defaultdict(dict)

In [26]:
def mean_of_counter(counter):
    # Extract values from the Counter and calculate the mean
    values = list(counter.values())
    return sum(values) / len(values) if values else 0  # Prevent division by zero

In [27]:
def adjust_lightness(color, amount=0.5):
    """
    Adjusts the lightness of the given color by the provided amount.
    :param color: Input color in some format that matplotlib's `to_rgb` can handle.
    :param amount: Amount to adjust. > 1 for lighter, < 1 for darker.
    :return: Adjusted color.
    """
    import colorsys
    c = mcolors.to_rgb(color)
    h, l, s = colorsys.rgb_to_hls(*c)
    return colorsys.hls_to_rgb(h, max(0, min(1, amount * l)), s)

In [28]:
def generate_gradient_colors(base_color, num_colors=10):
    """
    Generate gradient colors based on a base color.
    
    :param base_color: The base color to create a gradient from.
    :param num_colors: The number of gradient colors to generate.
    :return: List of colors.
    """
    # Convert base color to HLS (Hue, Lightness, Saturation)
    h, l, s = colorsys.rgb_to_hls(*mcolors.to_rgb(base_color))

    # Determine the step size for lightness
    middle_index = num_colors // 2
    lightness_step = l / (middle_index if middle_index > 0 else 1)

    colors = []
    for i in range(num_colors):
        if i < middle_index:
            # For the lighter colors (before the middle)
            new_lightness = l + (middle_index - i) * lightness_step
        elif i == middle_index:
            # The middle color remains the base color
            new_lightness = l
        else:
            # For the darker colors (after the middle)
            new_lightness = l - (i - middle_index) * lightness_step

        # Ensure new lightness is within the valid range [0, 1]
        new_lightness = max(0, min(new_lightness, 1))

        new_color = colorsys.hls_to_rgb(h, new_lightness, s)
        colors.append(new_color)

    return colors

In [29]:
def get_significance_stars_from_p_value(p_value, number_of_comparisons=3):
    """
    """
    if p_value <= 0.001 / number_of_comparisons:
        return "***"
    elif p_value <= 0.01 / number_of_comparisons:
        return "***"
    elif p_value <= 0.05 / number_of_comparisons:
        return "*"
    else:
        return ""

In [30]:
def filter_spike_times(arr_2d, start, stop, padding=-1):
    """
    Filters a 2D array based on a start and stop condition and pads shorter arrays to match the length of the longest array.

    Parameters:
    arr_2d (numpy.ndarray): 2D array to be filtered and padded.
    start (int or float): Lower bound of the condition.
    stop (int or float): Upper bound of the condition.
    padding (int or float): Value to use for padding, defaults to -1.

    Returns:
    numpy.ndarray: A 2D array where each inner array has been filtered based on the condition and padded to match the length of the longest array.
    """

    # Filter each row in arr_2d based on the condition
    masked_data = [row[(row >= start) & (row < stop)] for row in arr_2d]

    # Determine the maximum length of arrays in the list
    max_length = max(len(arr) for arr in masked_data)

    # Pad each array in masked_data with the padding value so they all have the same length (max_length)
    padded_arrays = [np.concatenate([x, np.full([max_length - len(x)], padding)]) for x in masked_data]

    # Convert the list of arrays to a 2D numpy array
    padded_arrays = np.array(padded_arrays)

    return padded_arrays

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [31]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [32]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject',
       ...
       'reward_port', 'box_bottom_left', 'box_bottom_right', 'agent',
       'subject_locations', 'agent_locations', 'subject_thorax_velocity',
       'agent_thorax_velocity', 'subject_thorax_to_reward_port',
       'agent_thorax_to_reward_port'],
      dtype='object', length=168)

In [33]:
TRIAL_LABELS_DF.columns

Index(['tracked_subject', 'box_number', 'sleap_name', 'video_name',
       'current_subject', 'tone_start_frame', 'reward_start_frame',
       'tone_stop_frame', 'condition', 'competition_closeness', 'notes',
       'experiment', 'session_dir', 'all_subjects', 'tone_start_timestamp',
       'tone_stop_timestamp', 'trial_label'],
      dtype='object')

## Merging the labels and LFP spectral

In [34]:
TRIAL_LABELS_DF.head()

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,notes,experiment,session_dir,all_subjects,tone_start_timestamp,tone_stop_timestamp,trial_label
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",1126742,1326741,win
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1125,1225,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",1126742,1326741,lose
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",3526740,3726740,lose
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.2,3519,3619,3720,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",3526740,3726740,win
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",5826740,6026737,lose


In [35]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,reward_port,box_bottom_left,box_bottom_right,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1.1,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
1,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 930], [930, 948...","[[32974, 32976], [33201, 33207], [33208, 33211...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1.1,...,"[48.01956210909252, 37.264554576109816]","[33.1534452699471, 35.61101869926864]","[62.60288905131385, 35.71974322144712]",None,"[[[27.67316905433137, 31.06298795228111], [27....",NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,"[21.36, 21.36, 21.36, 21.36, 21.36, 21.36, 21....",NaN
2,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1.2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",None,"[[[34.36404036584844, 35.615553316175784], [34...",NaN,"[55.4, 52.03, 48.72, 45.53, 42.44, 39.44, 36.5...",NaN,"[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...",NaN
3,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,1.2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.1,"[[[34.36404036584844, 35.615553316175784], [34...","[[[46.68890098287189, 11.495085446056166], [43...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1..."
4,2,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...","[[33601, 33798], [34108, 34165], [34166, 34179...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,1.1,...,"[45.31189572048577, 36.38317068198775]","[30.490085836306232, 34.91627159792731]","[60.10744182471764, 34.262899786598894]",None,"[[[46.71964110607613, 36.493379374373234], [44...",NaN,"[5.8, 5.207, 4.64, 4.1, 3.596, 3.127, 2.7, 2.3...",NaN,"[4.633, 4.09, 4.36, 4.656, 4.65, 4.383, 4.35, ...",NaN


In [36]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [37]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)
TRIAL_LABELS_DF["current_subject"] = TRIAL_LABELS_DF["current_subject"].astype(str)

In [38]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.3',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.2',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.2',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.1',
       '20230620_114347_standard_comp_

In [39]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.3',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.2',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.2',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.3',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.2',
       '20230619_115321_standard_comp_

In [40]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["current_subject", "video_name"], how="inner", suffixes=('', '_y'))

In [41]:
# TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col], errors="ignore")

In [42]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "_y" in col], errors="ignore")

In [43]:
TRIALS_AND_SPECTRAL_DF.head()

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,...,reward_port,box_bottom_left,box_bottom_right,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,Subj 2 Only,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,Subj 2 Only,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."


In [44]:
TRIALS_AND_SPECTRAL_DF.columns

Index(['tracked_subject', 'box_number', 'sleap_name', 'video_name',
       'current_subject', 'tone_start_frame', 'reward_start_frame',
       'tone_stop_frame', 'condition', 'competition_closeness',
       ...
       'reward_port', 'box_bottom_left', 'box_bottom_right', 'agent',
       'subject_locations', 'agent_locations', 'subject_thorax_velocity',
       'agent_thorax_velocity', 'subject_thorax_to_reward_port',
       'agent_thorax_to_reward_port'],
      dtype='object', length=178)

In [45]:
TRIALS_AND_SPECTRAL_DF

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,...,reward_port,box_bottom_left,box_bottom_right,agent,subject_locations,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,Subj 2 Only,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,Subj 2 Only,...,"[45.23441700698972, 35.83828707637651]","[30.458913657178506, 34.21867013031604]","[59.89132415244771, 34.13310301149612]",1.2,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...","[1.8125, 1.707, 1.6045, 1.506, 1.409, 1.315, 1...","[14.336, 12.875, 11.47, 10.11, 8.81, 7.57, 6.3...","[26.45, 26.16, 26.1, 25.77, 25.72, 25.78, 26.1...","[6.11, 6.06, 6.125, 6.566, 6.605, 6.676, 7.13,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364,"['1.2', '2.2']",2,20230630_115506_standard_comp_to_novel_agent_D...,20230630_115506_standard_comp_to_novel_agent_D...,1.2,60714,60814,60914,2.2,After trial,...,"[46.21123795031396, 36.95847031673951]","[31.450146241148413, 35.66393919018587]","[61.147866209397755, 35.553202690286255]",2.2,"[[[4.371455668449597, 33.21801163507191], [58....","[[[32.683056182830434, 34.92733973755609], [32...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0756, 0.0727, 0.0699, 0.06714, 0.0644, 0.06...","[41.38, 41.38, 41.38, 41.38, 41.38, 41.38,

# Calculating the relevant timestamps

In [46]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 30 * 20000

In [47]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 20 * 20000

In [48]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

0     526742
1    2926740
2    5226740
3    7026736
4    8126734
Name: baseline_start_timestamp, dtype: int64

In [49]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

0     726742
1    3126740
2    5426740
3    7226736
4    8326734
Name: baseline_stop_timestamp, dtype: int64

In [50]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1126742
1    3526740
2    5826740
3    7626736
4    8726734
Name: tone_start_timestamp, dtype: int64

In [51]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    1326741
1    3726740
2    6026737
3    7826735
4    8926734
Name: tone_stop_timestamp, dtype: int64

# Filtering out the SLEAP posed for during trials

In [52]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [53]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head()

0    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
1    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
2    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
3    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
4    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
Name: video_frame, dtype: object

In [54]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "locations" in col or "velocity" in col or "to_reward_port" in col or "video_frame" in col]

In [55]:
sleap_columns

['subject_locations',
 'agent_locations',
 'subject_thorax_velocity',
 'agent_thorax_velocity',
 'subject_thorax_to_reward_port',
 'agent_thorax_to_reward_port',
 'video_frame']

In [56]:
TRIALS_AND_SPECTRAL_DF["video_timestamps"].iloc[0].shape

(68258,)

In [57]:
TRIALS_AND_SPECTRAL_DF["video_frame"].iloc[0].shape

(68258,)

In [58]:
TRIALS_AND_SPECTRAL_DF["subject_thorax_to_reward_port"].iloc[0].shape

(68258,)

In [59]:
for col in sorted(sleap_columns):
    updated_item_col = "baseline_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "baseline_video_timestamps".format(col)
    if "agent" in col:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if x["agent"] else np.nan, axis=1)
    else:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0], axis=1)

baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baseline_subject_locations
baseline_subject_thorax_to_reward_port
baseline_subject_thorax_velocity
baseline_video_frame


In [60]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_video_timestamps".format(col)
    if "agent" in col:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if x["agent"] else np.nan, axis=1)
    else:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0], axis=1)

trial_agent_locations
trial_agent_thorax_to_reward_port
trial_agent_thorax_velocity
trial_subject_locations
trial_subject_thorax_to_reward_port
trial_subject_thorax_velocity
trial_video_frame


In [61]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["trial_subject_thorax_velocity"].apply(lambda x: np.isnan(x).any())]["baseline_subject_locations"]

Series([], Name: baseline_subject_locations, dtype: object)

In [62]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["video_timestamps"], errors="ignore")

In [63]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

tracked_subject
box_number
sleap_name
video_name
current_subject
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band
mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
mPFC_RMS_filtered_power_all_frequencies_all_windows
MD_RMS_filtered_power_all_frequencies_all_windows
LH_RMS_filtered_power_all_frequencies_all_windows
BLA_RMS_filtered_power_all_frequencies_all_windows
vHPC_RMS_filtered_power_all

- Filtering coherence

In [64]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [65]:
coherence_columns

['BLA_LH_RMS_filtered_coherence_all_frequencies_all_windows',
 'BLA_MD_RMS_filtered_coherence_all_frequencies_all_windows',
 'BLA_mPFC_RMS_filtered_coherence_all_frequencies_all_windows',
 'BLA_vHPC_RMS_filtered_coherence_all_frequencies_all_windows',
 'LH_MD_RMS_filtered_coherence_all_frequencies_all_windows',
 'LH_mPFC_RMS_filtered_coherence_all_frequencies_all_windows',
 'LH_vHPC_RMS_filtered_coherence_all_frequencies_all_windows',
 'MD_mPFC_RMS_filtered_coherence_all_frequencies_all_windows',
 'MD_vHPC_RMS_filtered_coherence_all_frequencies_all_windows',
 'mPFC_vHPC_RMS_filtered_coherence_all_frequencies_all_windows',
 'BLA_LH_coherence_theta',
 'BLA_LH_coherence_gamma',
 'BLA_MD_coherence_theta',
 'BLA_MD_coherence_gamma',
 'BLA_mPFC_coherence_theta',
 'BLA_mPFC_coherence_gamma',
 'BLA_vHPC_coherence_theta',
 'BLA_vHPC_coherence_gamma',
 'LH_MD_coherence_theta',
 'LH_MD_coherence_gamma',
 'LH_mPFC_coherence_theta',
 'LH_mPFC_coherence_gamma',
 'LH_vHPC_coherence_theta',
 'LH_vHPC_

In [66]:
for col in coherence_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_coherence_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

BLA_LH_RMS_filtered_coherence_all_frequencies
BLA_MD_RMS_filtered_coherence_all_frequencies
BLA_mPFC_RMS_filtered_coherence_all_frequencies
BLA_vHPC_RMS_filtered_coherence_all_frequencies
LH_MD_RMS_filtered_coherence_all_frequencies
LH_mPFC_RMS_filtered_coherence_all_frequencies
LH_vHPC_RMS_filtered_coherence_all_frequencies
MD_mPFC_RMS_filtered_coherence_all_frequencies
MD_vHPC_RMS_filtered_coherence_all_frequencies
mPFC_vHPC_RMS_filtered_coherence_all_frequencies
BLA_LH_coherence_theta
BLA_LH_coherence_gamma
BLA_MD_coherence_theta
BLA_MD_coherence_gamma
BLA_mPFC_coherence_theta
BLA_mPFC_coherence_gamma
BLA_vHPC_coherence_theta
BLA_vHPC_coherence_gamma
LH_MD_coherence_theta
LH_MD_coherence_gamma
LH_mPFC_coherence_theta
LH_mPFC_coherence_gamma
LH_vHPC_coherence_theta
LH_vHPC_coherence_gamma
MD_mPFC_coherence_theta
MD_mPFC_coherence_gamma
MD_vHPC_coherence_theta
MD_vHPC_coherence_gamma
mPFC_vHPC_coherence_theta
mPFC_vHPC_coherence_gamma


In [67]:
for col in coherence_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_coherence_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

BLA_LH_RMS_filtered_coherence_all_frequencies
BLA_MD_RMS_filtered_coherence_all_frequencies
BLA_mPFC_RMS_filtered_coherence_all_frequencies
BLA_vHPC_RMS_filtered_coherence_all_frequencies
LH_MD_RMS_filtered_coherence_all_frequencies
LH_mPFC_RMS_filtered_coherence_all_frequencies
LH_vHPC_RMS_filtered_coherence_all_frequencies
MD_mPFC_RMS_filtered_coherence_all_frequencies
MD_vHPC_RMS_filtered_coherence_all_frequencies
mPFC_vHPC_RMS_filtered_coherence_all_frequencies
BLA_LH_coherence_theta
BLA_LH_coherence_gamma
BLA_MD_coherence_theta
BLA_MD_coherence_gamma
BLA_mPFC_coherence_theta
BLA_mPFC_coherence_gamma
BLA_vHPC_coherence_theta
BLA_vHPC_coherence_gamma
LH_MD_coherence_theta
LH_MD_coherence_gamma
LH_mPFC_coherence_theta
LH_mPFC_coherence_gamma
LH_vHPC_coherence_theta
LH_vHPC_coherence_gamma
MD_mPFC_coherence_theta
MD_mPFC_coherence_gamma
MD_vHPC_coherence_theta
MD_vHPC_coherence_gamma
mPFC_vHPC_coherence_theta
mPFC_vHPC_coherence_gamma


In [68]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [69]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [70]:
granger_columns

['BLA_LH_granger_all_frequencies_all_windows',
 'LH_BLA_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'MD_BLA_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'mPFC_BLA_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'vHPC_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'MD_LH_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'mPFC_LH_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'vHPC_LH_granger_all_frequencies_all_windows',
 'MD_mPFC_granger_all_frequencies_all_windows',
 'mPFC_MD_granger_all_frequencies_all_windows',
 'MD_vHPC_granger_all_frequencies_all_windows',
 'vHPC_MD_granger_all_frequencies_all_windows',
 'mPFC_vHPC_granger_all_frequencies_all_windows',
 'vHPC_mPFC_granger_all_frequencies_all_windows',
 'BLA-to-LH_granger_theta',
 'BLA-to-LH_

In [71]:
for col in granger_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_granger_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

BLA_LH_granger_all_frequencies
LH_BLA_granger_all_frequencies
BLA_MD_granger_all_frequencies
MD_BLA_granger_all_frequencies
BLA_mPFC_granger_all_frequencies
mPFC_BLA_granger_all_frequencies
BLA_vHPC_granger_all_frequencies
vHPC_BLA_granger_all_frequencies
LH_MD_granger_all_frequencies
MD_LH_granger_all_frequencies
LH_mPFC_granger_all_frequencies
mPFC_LH_granger_all_frequencies
LH_vHPC_granger_all_frequencies
vHPC_LH_granger_all_frequencies
MD_mPFC_granger_all_frequencies
mPFC_MD_granger_all_frequencies
MD_vHPC_granger_all_frequencies
vHPC_MD_granger_all_frequencies
mPFC_vHPC_granger_all_frequencies
vHPC_mPFC_granger_all_frequencies
BLA-to-LH_granger_theta
BLA-to-LH_granger_gamma
LH-to-BLA_granger_theta
LH-to-BLA_granger_gamma
BLA-to-MD_granger_theta
BLA-to-MD_granger_gamma
MD-to-BLA_granger_theta
MD-to-BLA_granger_gamma
BLA-to-mPFC_granger_theta
BLA-to-mPFC_granger_gamma
mPFC-to-BLA_granger_theta
mPFC-to-BLA_granger_gamma
BLA-to-vHPC_granger_theta
BLA-to-vHPC_granger_gamma
vHPC-to-BLA_

In [72]:
for col in granger_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_granger_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

BLA_LH_granger_all_frequencies
LH_BLA_granger_all_frequencies
BLA_MD_granger_all_frequencies
MD_BLA_granger_all_frequencies
BLA_mPFC_granger_all_frequencies
mPFC_BLA_granger_all_frequencies
BLA_vHPC_granger_all_frequencies
vHPC_BLA_granger_all_frequencies
LH_MD_granger_all_frequencies
MD_LH_granger_all_frequencies
LH_mPFC_granger_all_frequencies
mPFC_LH_granger_all_frequencies
LH_vHPC_granger_all_frequencies
vHPC_LH_granger_all_frequencies
MD_mPFC_granger_all_frequencies
mPFC_MD_granger_all_frequencies
MD_vHPC_granger_all_frequencies
vHPC_MD_granger_all_frequencies
mPFC_vHPC_granger_all_frequencies
vHPC_mPFC_granger_all_frequencies
BLA-to-LH_granger_theta
BLA-to-LH_granger_gamma
LH-to-BLA_granger_theta
LH-to-BLA_granger_gamma
BLA-to-MD_granger_theta
BLA-to-MD_granger_gamma
MD-to-BLA_granger_theta
MD-to-BLA_granger_gamma
BLA-to-mPFC_granger_theta
BLA-to-mPFC_granger_gamma
mPFC-to-BLA_granger_theta
mPFC-to-BLA_granger_gamma
BLA-to-vHPC_granger_theta
BLA-to-vHPC_granger_gamma
vHPC-to-BLA_

In [73]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

In [74]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

- Filtering power

In [75]:
TRIALS_AND_SPECTRAL_DF.head()

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,...,trial_mPFC-to-MD_granger_gamma,trial_MD-to-vHPC_granger_theta,trial_MD-to-vHPC_granger_gamma,trial_vHPC-to-MD_granger_theta,trial_vHPC-to-MD_granger_gamma,trial_mPFC-to-vHPC_granger_theta,trial_mPFC-to-vHPC_granger_gamma,trial_vHPC-to-mPFC_granger_theta,trial_vHPC-to-mPFC_granger_gamma,trial_granger_timestamps
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,...,"[0.2743068, 0.1816457, 0.20403126, 0.39949384,...","[0.4503957, 0.5514915, 0.20297746, 0.3768568, ...","[0.35033533, 0.25229087, 0.23366664, 0.3120289...","[0.3872143, 0.48870432, 0.45020568, 0.61337423...","[0.48877043, 0.22785409, 0.13890521, 0.265355,...","[0.48188347, 0.7108543, 0.18493713, 0.4764926,...","[0.34150958, 0.16291973, 0.20073977, 0.2023801...","[0.13133682, 0.23562104, 0.2440842, 0.15778197...","[0.35235053, 0.17450492, 0.37754112, 0.2303077...","[1130000, 1140000, 1150000, 1160000, 1170000, ..."
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,...,"[0.19837809, 0.5073394, 0.2601421, 0.17640117,...","[nan, 0.4103218, 0.1676341, nan, nan, 0.165973...","[nan, 0.41292667, 0.30698562, nan, nan, 0.3141...","[nan, 0.49083915, 0.09444503, nan, nan, 0.4753...","[nan, 0.24761999, 0.24060199, nan, nan, 0.3022...","[nan, 0.77709764, 0.46270016, nan, nan, 0.1674...","[nan, 0.4095989, 0.7093766, nan, nan, 0.181099...","[nan, 0.5647509, 0.45051277, nan, nan, 0.26605...","[nan, 0.18905884, 0.20349285, nan, nan, 0.2695...","[3530000, 3540000, 3550000, 3560000, 3570000, ..."
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,...,"[0.2451308, 0.22220398, 0.42297348, 0.21880822...","[0.3611453, 0.15535778, 0.10153123, 0.1033775,...","[0.28545213, 0.2011751, 0.30536118, 0.27875093...","[0.57998323, 0.70480734, 0.6927041, 0.79001415...","[0.26702482, 0.20771386, 0.6002271, 0.34293473...","[0.12613265, 0.34686637, 0.37530336, 0.0726977...","[0.2384008, 0.3289804, 0.38966492, 0.16582075,...","[0.33719257, 0.35507208, 0.390221, 0.41154325,...","[0.37617797, 0.284876, 0.4089172, 0.16548316, ...","[5830000, 5840000, 5850000, 5860000, 5870000, ..."
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,Subj 2 Only,...,"[0.19989435, 0.3218986, 0.1846013, 0.44848886,...","[0.5495982, 0.15429461, 0.13154985, 0.21991509...","[0.43715674, 0.35011134, 0.31304544, 0.394448,...","[0.8319382, 0.8078395, 0.58001375, 0.2841426, ...","[0.35010985, 0.21684945, 0.14930977, 0.1355896...","[0.09518096, 0.20900358, 0.18624677, 0.2759102...","[0.1536734, 0.27427787, 0.33714557, 0.3097984,...","[0.49204752, 0.7473691, 0.5807837, 0.09881899,...","[0.3218193, 0.21725373, 0.33039397, 0.4509097,...","[7630000, 7640000, 7650000, 7660000, 7670000, ..."
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,Subj 2 Only,...,"[0.25939098, 0.21315253, 0.46966016, 0.1842520...","[0.46183974, 0.16732047, 0.38939884, 0.481716,...","[0.3225777, 0.45540878, 0.3549677, 0.51858795,...","[0.14207345, 1.1679717, 1.0912386, 0.24064888,...","[0.24173824, 0.27463642, 0.2826689, 0.22011085...","[0.1101417, 0.14950544, 0.5411582, 0.23466948,...","[0.26624367, 0.34639746, 0.4196817, 0.29253903...","[0.21166928, 0.93035126, 0.45488098, 0.3350755...","[0.21460077, 0.40119368, 0.21195264, 0.1034163...","[8730000, 8740000, 8750000, 8760000, 8770000, ..."


In [76]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [77]:
power_columns

['mPFC_RMS_filtered_power_all_frequencies_all_windows',
 'MD_RMS_filtered_power_all_frequencies_all_windows',
 'LH_RMS_filtered_power_all_frequencies_all_windows',
 'BLA_RMS_filtered_power_all_frequencies_all_windows',
 'vHPC_RMS_filtered_power_all_frequencies_all_windows',
 'mPFC_RMS_filtered_power_theta',
 'mPFC_RMS_filtered_power_gamma',
 'MD_RMS_filtered_power_theta',
 'MD_RMS_filtered_power_gamma',
 'LH_RMS_filtered_power_theta',
 'LH_RMS_filtered_power_gamma',
 'BLA_RMS_filtered_power_theta',
 'BLA_RMS_filtered_power_gamma',
 'vHPC_RMS_filtered_power_theta',
 'vHPC_RMS_filtered_power_gamma']

In [78]:
for col in power_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_power_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)

mPFC_RMS_filtered_power_all_frequencies
MD_RMS_filtered_power_all_frequencies
LH_RMS_filtered_power_all_frequencies
BLA_RMS_filtered_power_all_frequencies
vHPC_RMS_filtered_power_all_frequencies
mPFC_RMS_filtered_power_theta
mPFC_RMS_filtered_power_gamma
MD_RMS_filtered_power_theta
MD_RMS_filtered_power_gamma
LH_RMS_filtered_power_theta
LH_RMS_filtered_power_gamma
BLA_RMS_filtered_power_theta
BLA_RMS_filtered_power_gamma
vHPC_RMS_filtered_power_theta
vHPC_RMS_filtered_power_gamma


In [79]:
for col in power_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_power_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)

mPFC_RMS_filtered_power_all_frequencies
MD_RMS_filtered_power_all_frequencies
LH_RMS_filtered_power_all_frequencies
BLA_RMS_filtered_power_all_frequencies
vHPC_RMS_filtered_power_all_frequencies
mPFC_RMS_filtered_power_theta
mPFC_RMS_filtered_power_gamma
MD_RMS_filtered_power_theta
MD_RMS_filtered_power_gamma
LH_RMS_filtered_power_theta
LH_RMS_filtered_power_gamma
BLA_RMS_filtered_power_theta
BLA_RMS_filtered_power_gamma
vHPC_RMS_filtered_power_theta
vHPC_RMS_filtered_power_gamma


In [80]:
TRIALS_AND_SPECTRAL_DF["baseline_power_timestamps"]

0       [530000, 540000, 550000, 560000, 570000, 58000...
1       [2930000, 2940000, 2950000, 2960000, 2970000, ...
2       [5230000, 5240000, 5250000, 5260000, 5270000, ...
3       [7030000, 7040000, 7050000, 7060000, 7070000, ...
4       [8130000, 8140000, 8150000, 8160000, 8170000, ...
                              ...                        
1364    [60080000, 60090000, 60100000, 60110000, 60120...
1365    [61580000, 61590000, 61600000, 61610000, 61620...
1366    [63580000, 63590000, 63600000, 63610000, 63620...
1367    [65480000, 65490000, 65500000, 65510000, 65520...
1368    [66480000, 66490000, 66500000, 66510000, 66520...
Name: baseline_power_timestamps, Length: 1369, dtype: object

In [81]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [82]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tracked_subject
box_number
sleap_name
video_name
current_subject
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band
mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
start_frame
stop_frame
in_video_subjects
body_parts
box_top_left
box_top_right
reward_port
box_bottom_left
box_bottom_right
agent
base

# Filtering out phase

In [83]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [84]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [85]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [86]:
phase_columns = phase_columns + band_columns +lfp_columns

In [87]:
phase_columns

['mPFC_theta_phase',
 'MD_theta_phase',
 'LH_theta_phase',
 'BLA_theta_phase',
 'vHPC_theta_phase',
 'mPFC_gamma_phase',
 'MD_gamma_phase',
 'LH_gamma_phase',
 'BLA_gamma_phase',
 'vHPC_gamma_phase',
 'mPFC_theta_band',
 'MD_theta_band',
 'LH_theta_band',
 'BLA_theta_band',
 'vHPC_theta_band',
 'mPFC_gamma_band',
 'MD_gamma_band',
 'LH_gamma_band',
 'BLA_gamma_band',
 'vHPC_gamma_band']

In [88]:
for col in phase_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_lfp_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)

mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band


In [89]:
for col in phase_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_lfp_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)

mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band


In [90]:
TRIALS_AND_SPECTRAL_DF["baseline_lfp_timestamps"]

0       [526760, 526780, 526800, 526820, 526840, 52686...
1       [2926740, 2926760, 2926780, 2926800, 2926820, ...
2       [5226740, 5226760, 5226780, 5226800, 5226820, ...
3       [7026740, 7026760, 7026780, 7026800, 7026820, ...
4       [8126740, 8126760, 8126780, 8126800, 8126820, ...
                              ...                        
1364    [60073460, 60073480, 60073500, 60073520, 60073...
1365    [61573480, 61573500, 61573520, 61573540, 61573...
1366    [63573520, 63573540, 63573560, 63573580, 63573...
1367    [65473540, 65473560, 65473580, 65473600, 65473...
1368    [66473540, 66473560, 66473580, 66473600, 66473...
Name: baseline_lfp_timestamps, Length: 1369, dtype: object

In [91]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [92]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [93]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tracked_subject
box_number
sleap_name
video_name
current_subject
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
start_frame
stop_frame
in_video_subjects
body_parts
box_top_left
box_top_right
reward_port
box_bottom_left
box_bottom_right
agent
baseline_start_timestamp
baseline_stop_timestamp
baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baseline_subject_locations
baseline_subject_thorax_to_reward_port
baseline_subject_thorax_velocity
baseline_video_frame
baseline_video_timestamps
trial_agent_locations
trial_agent_thorax_

# Filtering out spikes

In [95]:
raise ValueError()

ValueError: 

In [94]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"]).astype(int), axis=1)



KeyboardInterrupt



In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape

# Filtering for rows that are in the video

In [96]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_video_frame"][0] <= x["stop_frame"], axis=1)

In [97]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [98]:
TRIALS_AND_SPECTRAL_DF

,index,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,...,trial_LH_theta_band,trial_BLA_theta_band,trial_vHPC_theta_band,trial_mPFC_gamma_band,trial_MD_gamma_band,trial_LH_gamma_band,trial_BLA_gamma_band,trial_vHPC_gamma_band,trial_lfp_timestamps,in_video
0,0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,...,"[-0.1965594, -0.1906385, -0.1846614, -0.178647...","[-0.6278904, -0.6110051, -0.5924792, -0.572370...","[-0.86628234, -0.8033415, -0.73908955, -0.6737...","[-0.20077267, -0.27177534, -0.33255213, -0.378...","[0.042438794, -0.06269324, -0.16432026, -0.255...","[0.04900858, -0.041143205, -0.13162935, -0.216...","[-0.034259137, -0.07323087, -0.11172231, -0.14...","[0.091950886, 0.030192526, -0.039998747, -0.11...","[1126760, 1126780, 1126800, 1126820, 1126840, ...",True
1,1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,...,"[0.13085423, 0.18954182, 0.24687698, 0.3026803...","[0.23016661, 0.29850513, 0.36496305, 0.4293212...","[0.5561959, 0.5872191, 0.6153956, 0.6406603, 0...","[-0.20394863, -0.14480965, -0.0752808, -0.0018...","[-0.17063874, -0.1823823, -0.18196633, -0.1690...","[-0.13379243, -0.14699419, -0.15053916, -0.144...","[0.30445713, 0.2672235, 0.21840814, 0.16240521...","[0.41455534, 0.38904047, 0.34448272, 0.2843119...","[3526740, 3526760, 3526780, 3526800, 3526820, ...",True
2,2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,...,"[-0.117264144, -0.18494129, -0.25172737, -0.31...","[0.1510647, 0.102288105, 0.05333691, 0.0043895...","[0.15260983, 0.15515444, 0.15816545, 0.1616569...","[-0.2272427, -0.3109095, -0.375287, -0.4164722...","[-0.124851465, -0.1811587, -0.22630072, -0.257...","[-0.08437547, -0.14467272, -0.19493897, -0.231...","[-0.031209087, -0.07443029, -0.11561694, -0.15...","[-0.20006756, -0.29163465, -0.35794356, -0.393...","[5826740, 5826760, 5826780, 5826800, 5826820, ...",True
3,3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,...,"[0.571904, 0.57688946, 0.57978463, 0.5806063, ...","[1.093931, 1.0820043, 1.0669925, 1.0489453, 1....","[0.30607852, 0.26300424, 0.21877678, 0.1735299...","[-0.3201298, -0.3654427, -0.38641584, -0.38069...","[-0.22039057, -0.25678217, -0.2772813, -0.2792...","[-0.18128534, -0.2485002, -0.2972655, -0.32333...","[0.042083926, -0.04936665, -0.13645948, -0.211...","[0.16657004, 0.15574361, 0.14005733, 0.1210614...","[7626740, 7626760, 7626780, 7626800, 7626820, ...",True
4,4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,...,"[0.12631682, 0.12798116, 0.12979607, 0.1317504...","[-0.05114213, -0.075476125, -0.09903707, -0.12...","[-0.36225998, -0.3638156, -0.36478588, -0.3651...","[0.30167827, 0.2791766, 0.24319397, 0.19783372...","[0.16291839, 0.15559189, 0.14303625, 0.1267487...","[0.005076932, 0.04650006, 0.08811159, 0.127866...","[0.5704153, 0.55549455, 0.5117636, 0.44269294,...","[-0.14728308, -0.10256097, -0.05680496, -0.009...","[8726740, 8726760, 8726780, 8726800, 8726820, ...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,1364,"['1.2', '2.2']",2,20230630_115506_standard_comp_to_novel_agent_D...,20230630_115506_standard_comp_to_novel_agent_D...,1.2,60714,60814,60914,2.2,...,"[0.44909856, 0.4724813, 0.49443373, 0.51487184...","[0.3336614, 0.32942677, 0.32425243, 0.31815872...","[-0.36571392, -0.39123663, -0.41522902, -0.437...","[-0.019997114, -0.05471756, -0.08678063, -0.11...","[-0.15819319, -0.27901205, -0.3875353, -0.4765...","[0.016292952, -0.11031159, -0.23285471, -0

In [99]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()